In [1]:
import numpy as np
import pandas as pd
import re
import os
import sys

In [2]:
from fuzzywuzzy import process, fuzz
def fuzzy_match_and_score(string,array):
    fuzzy_extraction = process.extractOne(string,array)
    return fuzzy_extraction[0], fuzzy_extraction[1]

/usr/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [79]:
def read_set_in_csv(x):
    if x=='set()':
        return set()
    else:
        return set(x.replace("'",'').strip('{}').split(', '))

MATCHING IUCN->CHECKLIST->HERBIER

Chargement de l'IUCN

Chargement de la checklist

In [4]:
checklist = pd.read_csv('/home/joon/data/checklist_taxonomy_preprocessed.csv',index_col='plant_name_id')
checklist = checklist[['taxon_status', 'family', 'genus', 'species', 'taxon_name', 'taxon_authors', 'accepted_plant_name_id']]
checklist.dropna(axis=0,subset=['species'],inplace=True)

Chargement de l'herbier

In [5]:
herbier = pd.read_csv('/home/joon/data/herbier.csv',sep=',',encoding='iso-8859-1')
herbier.AUT_COM_EP.replace(np.nan,'',inplace=True)

Récupération du matching iucn -> checklist

In [80]:
iucn_checklist_matching = pd.read_csv('/home/joon/data/iucn-checklist-matching.csv',index_col=0,header=None,squeeze=True,converters={1:read_set_in_csv})

In [7]:
accepted_plant_name_ids = checklist.accepted_plant_name_id.unique()

In [8]:
checklist = checklist[checklist.accepted_plant_name_id.isin(accepted_plant_name_ids)]

# Matching genus

In [9]:
herbier = herbier[(herbier.GENRE.isnull() == False) & (herbier.GENRE.isnull() == False)]

herbier_genus_values = herbier.GENRE.unique()
checklist_genus_values = checklist.genus.unique()

In [102]:
genus_checklist_herbier_matching = pd.Series([])

for genus in checklist_genus_values:
    if genus.lower() in herbier_genus_values:
        genus_checklist_herbier_matching[genus] = genus.lower()
    else:
        considered_values = [value for value in herbier_genus_values if ((len(genus)*3/4-2<=len(value)) & (len(value) <=len(genus)*4/3+2) & (value[0].lower()==genus[0].lower()))]
        if len(considered_values) >= 1:
            genus_fuzzy_matchings = pd.DataFrame([[value,score] for (value, score) in process.extract(genus,considered_values)] ,columns=['value','score'])
            genus_fuzzy_matchings = genus_fuzzy_matchings[genus_fuzzy_matchings.score==genus_fuzzy_matchings.score.max()]
            genus_checklist_herbier_matching[genus] = genus_fuzzy_matchings.value.values[0]

In [112]:
genus_checklist_herbier_matching.to_csv('genus_checklist_herbier_matching.csv',header=False)

In [115]:
genus_checklist_herbier_matching = pd.read_csv('genus_checklist_herbier_matching.csv',index_col=0,header=None)

# Matching checklist rows ->herbier TAXONK

In [258]:
def checklist_herbier_matching_process_genus(genus):
    global genus_checklist_herbier_matching, checklist, herbier

    checklist_herbier_matching = pd.Series([])

    genus_values = genus_checklist_herbier_matching.loc[genus]
    checklist_species_from_genus = checklist.species[checklist.genus==genus]

    search_res_g = herbier[herbier.GENRE.isin(genus_values)] # filter by genus only

    for plant_name_id in checklist_species_from_genus.index:
        if search_res_g.shape[0]==0:
            checklist_herbier_matching[plant_name_id]={}
        else:
            giveup = False
            checklist_row = checklist.loc[plant_name_id]
            species = checklist_row.species
            g_s = genus + ' ' + species
            taxon = checklist_row.taxon_name
            var_subsp = taxon.replace(g_s,'').replace(' var. ','').replace(' subsp. ','')
            authority = checklist_row.taxon_authors

            search_res_gs = search_res_g[search_res_g.ESPECE==species]

            if var_subsp != '':
                search_res_gs = search_res_gs[search_res_gs.NOM_INFSP==var_subsp]
            else:
                search_res_gs = search_res_gs[search_res_gs.NOM_INFSP.isnull()]

            if search_res_gs.shape[0] == 0: # if match fails with genus+species, consider all genus, and fuzzy match the species

                os.system('echo "'+str(plant_name_id)+"' > log.txt")
                name_fuzzy_matchings = None
                try:
                    name_fuzzy_matchings = pd.DataFrame([[name,score] for (name,score) in process.extract(species,search_res_g.ESPECE.unique(),limit=100)],columns=['name','score'])
                except:
                    print(sys.exc_info()[0])
                    print(g_s)
                name_fuzzy_matchings = name_fuzzy_matchings[name_fuzzy_matchings.score>=max(100-2*(100-name_fuzzy_matchings.score.max()),79)]
                search_res_gs = search_res_g[search_res_g.ESPECE.isin(name_fuzzy_matchings.name.values)] 

            if search_res_gs.shape[0] == 0: # if match fails (even fuzzy) with genus+species
                final_search_res = search_res_gs
                giveup = True

            if pd.isnull(authority):
                authority = ""
            authority = authority.strip()

            search_res_matching_authority = search_res_gs[search_res_gs.AUT_COM_EP==authority] # try exact match of author name

            if search_res_matching_authority.shape[0] == 0:
                authority_complete = authority
                authority = re.sub('\(.+\)','',authority).strip()
                search_res_matching_authority = search_res_gs[search_res_gs.AUT_COM_EP==authority] # try exact match of author name

            if search_res_matching_authority.TAXONPK.nunique() >= 1:
                final_search_res = search_res_matching_authority
            else:
                search_res_containing_authority = search_res_gs[search_res_gs.AUT_COM_EP.str.contains(authority)]
                if search_res_containing_authority.AUT_COM_EP.nunique() == 1:
                    final_search_res = search_res_containing_authority
                elif not giveup:    
                    if search_res_containing_authority.TAXONPK.nunique() >= 2:
                        search_res_gs = search_res_containing_authority
                    name_fuzzy_matchings = pd.DataFrame([[name,score] for (name, score) in process.extract(authority,search_res_gs.AUT_COM_EP.values,scorer=fuzz.partial_ratio,limit=100)] ,columns=['name','score'])
                    name_fuzzy_matchings = name_fuzzy_matchings[name_fuzzy_matchings.score>=max(100-2*(100-name_fuzzy_matchings.score.max()),80)]
                    if name_fuzzy_matchings.shape[0] > 1:
                        name_fuzzy_matchings = pd.DataFrame([[name,score] for (name, score) in process.extract(authority_complete,search_res_gs.AUT_COM_EP.values,scorer=fuzz.partial_ratio,limit=100)] ,columns=['name','score'])
                        name_fuzzy_matchings = name_fuzzy_matchings[name_fuzzy_matchings.score>=max(100-2*(100-name_fuzzy_matchings.score.max()),80)]
                    final_search_res = search_res_gs[search_res_gs.AUT_COM_EP.isin(name_fuzzy_matchings.name.values)] 

            # if final_search_res.TAXONPK.nunique() > 1:
            #     print(g_s,authority, 'nonunique matches. Authors:',final_search_res.AUT_COM_EP.unique())

            checklist_herbier_matching[plant_name_id]=set(list(final_search_res.TAXONPK.unique()))
    checklist_herbier_matching.to_csv('/home/joon/data/checklist-herbier-matching-'+genus+'.csv',header=False)

In [259]:
os.system('rm /home/joon/data/checklist-herbier-matching-*.csv')
os.system('rm /home/joon/data/checklist-herbier-matching.csv')

from multiprocessing import Pool
with Pool(processes=11) as pool:
    pool.map(checklist_herbier_matching_process_genus, checklist_genus_values)

#fusionner les résultats et supprimer les fichiers temporaires!
os.system('cat /home/joon/data/checklist-herbier-matching-*.csv > /home/joon/data/checklist-herbier-matching.csv')
os.system('rm /home/joon/data/checklist-herbier-matching-*.csv')

0

In [10]:
checklist_herbier_matching = pd.read_csv('/home/joon/data/checklist-herbier-matching.csv',index_col=0,header=None,squeeze=True,converters={1:read_set_in_csv})

In [ ]:
# checklist_herbier_matching = pd.DataFrame([],dtype=int)

# for genus in checklist_genus_values:
#     genus_values = genus_checklist_herbier_matching.query('genus_in_checklist==\''+genus+'\'')['matched_genus_in_herbier']
#     checklist_species_from_genus = checklist.species[checklist.genus==genus]

#     search_res_g = herbier[herbier.GENRE.isin(genus_values)] # filter by genus only

#     for index in checklist_species_from_genus.index:
#         giveup = False
#         checklist_row = checklist.loc[index]
#         species = checklist_row.species
#         g_s = genus + ' ' + species
#         taxon = checklist_row.taxon_name
#         var_subsp = taxon.replace(g_s,'').replace(' var. ','').replace(' subsp. ','')
#         authority = checklist_row.taxon_authors

#         search_res_gs = search_res_g[search_res_g.ESPECE==species]

#         if var_subsp != '':
#             search_res_gs = search_res_gs[search_res_gs.NOM_INFSP==var_subsp]
#         else:
#             search_res_gs = search_res_gs[search_res_gs.NOM_INFSP.isnull()]

#         if search_res_gs.shape[0] == 0: # if match fails with genus+species
#             final_search_res = search_res_gs
#             giveup = True

#         if pd.isnull(authority):
#             authority = ""

#         search_res_matching_authority = search_res_gs[search_res_gs.AUT_COM_EP==authority] # try exact match of author name

#         if search_res_matching_authority.TAXONPK.nunique() >= 1:
#             final_search_res = search_res_matching_authority
#         elif not giveup:
#             if search_res_containing_authority.TAXONPK.nunique() >= 2:
#                 search_res_gs = search_res_containing_authority
#             name_fuzzy_matchings = pd.DataFrame([[name,score] for (name, score) in process.extract(authority,search_res_gs.AUT_COM_EP.values,scorer=fuzz.partial_ratio,limit=100)] ,columns=['name','score'])
#             name_fuzzy_matchings = name_fuzzy_matchings[name_fuzzy_matchings.score>=max(100-2*(100-name_fuzzy_matchings.score.max()),80)]
#             final_search_res = search_res_gs[search_res_gs.AUT_COM_EP.isin(name_fuzzy_matchings.name.values)] 

#         if final_search_res.TAXONPK.nunique() > 1:
#             print(g_s,authority, 'nonunique matches. Authors:',final_search_res.AUT_COM_EP.unique())
#         # REPRENDRE
#         plant_name_id = int(checklist_row.plant_name_id)

#         for taxonpk in final_search_res.TAXONPK.unique():
#             checklist_herbier_matching = checklist_herbier_matching.append(pd.Series([plant_name_id,taxonpk]),ignore_index=True)

# checklist_herbier_matching.columns = ['checklist_plant_name_id','herbier_taxonpk']

In [337]:
#checklist_herbier_matching.astype(int).to_csv('checklist_herbier_matching.csv')

In [9]:
# checklist_herbier_matching = pd.read_csv('checklist_herbier_matching.csv',index_col=0)

In [82]:
checklist_accepted_herbier_taxonpk_matching = pd.Series([])

for accepted_plant_name_id in accepted_plant_name_ids:
    plant_name_ids = set(checklist[checklist.accepted_plant_name_id==accepted_plant_name_id].index)
    determinpks = {determinpk for foo in checklist_herbier_matching.loc[plant_name_ids] for determinpk in foo}
    taxonpks = set(herbier.TAXONPK[herbier.DETERMINPK.isin(determinpks)])
    checklist_accepted_herbier_taxonpk_matching[accepted_plant_name_id] = taxonpks

checklist_accepted_herbier_taxonpk_matching.to_csv('/home/joon/data/checklist-accepted-herbier-taxonpk-matching.csv',header=False)

In [11]:
checklist_accepted_herbier_taxonpk_matching = pd.read_csv('/home/joon/data/checklist-accepted-herbier-taxonpk-matching.csv',index_col=0,header=None,squeeze=True,converters={1:read_set_in_csv})

# Correspondance iucn index -> TAXONPK

In [273]:
iucn_checklist_matching = pd.read_csv('/home/joon/data/iucn-checklist-matching.csv',index_col=0,header=None,squeeze=True,converters={1:read_set_in_csv})

In [ ]:
iucn_herbier_taxonpk_matching = pd.Series([])

# for iucn_taxon_id in iucn_checklist_matching.loc[iucn_checklist_species_validated_matchings,'iucn_index'].unique():
for iucn_taxon_id in iucn_checklist_matching.index:
    accepted_plant_name_ids = iucn_checklist_matching.loc[iucn_taxon_id]
    taxonpks = {int(taxonpk) for foo in checklist_accepted_herbier_taxonpk_matching.reindex(accepted_plant_name_ids) for taxonpk in foo}
    iucn_herbier_taxonpk_matching[iucn_taxon_id] = taxonpks

iucn_herbier_taxonpk_matching.to_csv('/home/joon/data/iucn-herbier-taxonpk-matching.csv',header=False,index=True)